In [ ]:
# Instalar selenium
!pip install selenium

In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time

# Función para configurar el driver de Chrome con User-Agent
def setup_driver():
    chrome_options = Options()
    # Configura un User-Agent apropiado
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    chrome_options.add_argument(f"user-agent={user_agent}")

    # Inicia el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(10)
    driver.get("https://www.pap.hacienda.gob.es/bdnstrans/GE/es/concesiones/ayudas")
    return driver

# Configura el driver de Chrome
driver = setup_driver()

# Intenta cerrar el banner de cookies (si existe)
try:
    cookies_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.ID, "Autorizar"))
    )
    cookies_button.click()
    print("Banner de cookies cerrado.")
except Exception:
    print("No se encontró el banner de cookies o ya estaba cerrado.")

time.sleep(3)

# Selecciona el número de elementos mostrado por página (1000 elementos)
try:
    paginator_select = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "mat-select-2"))
    )
    
    # Desplaza la página hacia el elemento
    ActionChains(driver).move_to_element(paginator_select).perform()
    paginator_select.click()

    option_1000 = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='mat-option-text' and contains(text(), '1000')]"))
    )
    option_1000.click()
    print("Se ha seleccionado la opción de mostrar 1000 elementos por página.")
    
    time.sleep(20)  # Espera a que la página se actualice completamente
except Exception as e:
    print("Error al seleccionar el número de elementos por página:", str(e))

# Función para crear la carpeta "Datos" si no existe
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

# Función para ir a una página específica usando el selector "Ir a:"
def go_to_page(page_number):
    max_attempts = 100
    current_attempt = 0
    while current_attempt < max_attempts:
        try:
            # Abre el desplegable "Ir a:"
            page_select = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, "mat-select-0"))
            )

            # Desplaza la página hacia el elemento
            ActionChains(driver).move_to_element(page_select).perform()

            try:
                page_select.click()
            except:
                # Si falla, usa JavaScript
                driver.execute_script("arguments[0].click();", page_select)

            # Encuentra todas las opciones de página disponibles
            options = driver.find_elements(By.XPATH, "//span[@class='mat-option-text']")
            available_pages = [int(option.text) for option in options if option.text.isdigit()]
            available_pages.sort()

            # Comprueba si la página deseada está disponible
            if page_number in available_pages:
                option = driver.find_element(By.XPATH, f"//span[@class='mat-option-text' and text()='{page_number}']")
                option.click()
                print(f"Se ha navegado a la página {page_number}.")
                time.sleep(20)  # Espera a que la página se actualice completamente
                return
            else:
                # Selecciona la opción más grande disponible
                highest_page = max(available_pages)
                option = driver.find_element(By.XPATH, f"//span[@class='mat-option-text' and text()='{highest_page}']")
                option.click()
                print(f"Se ha navegado a la página {highest_page} para continuar.")
                time.sleep(20)  # Espera a que la página se actualice completamente
                current_attempt += 1
        except Exception as e:
            print(f"Error al navegar a la página {page_number}: {str(e)}")
            break

# Función para extraer los datos de la tabla con manejo de StaleElementReferenceException
def extract_data():
    retries = 3
    for attempt in range(retries):
        try:
            table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "body > app-root > main > app-ayudas > div:nth-child(1) > div:nth-child(2) > app-table-results > section > div.table-container.tabla-fija > table"))
            )
            rows = table.find_elements(By.TAG_NAME, "tr")
            page_data = []
            for row in rows[1:]:  # Ignora la cabecera
                columns = row.find_elements(By.TAG_NAME, "td")
                if columns:
                    row_data = [col.text for col in columns]
                    page_data.append(row_data)
            if page_data:
                return page_data
        except StaleElementReferenceException:
            print(f"StaleElementReferenceException: Reintentando... (Intento {attempt + 1}/{retries})")
            time.sleep(5)
        except TimeoutException:
            print("TimeoutException: No se pudo encontrar la tabla.")
            break
    print("No se pudieron extraer datos después de varios intentos.")
    return []

# Extrae los datos de un rango de páginas y guarda en CSV cada 10 páginas
def extract_range(start_page, end_page):
    global all_data
    all_data = []  # Definir all_data como una variable global para usarla fuera de la función
    create_folder("Datos")  # Crea la carpeta "Datos" si no existe
    for page in range(start_page, end_page + 1):
        data = extract_data()
        if data:
            all_data.extend(data)
            print(f"Página {page} completada con {len(data)} registros.")
        else:
            print(f"Sin datos en la página {page}, deteniendo la extracción.")
            break

        if (page - start_page + 1) % 10 == 0 or page == end_page:
            # Guarda los datos acumulados cada 10 páginas o al final del rango
            start_record = (page - 10 + 1) * 1000 if (page - start_page + 1) % 10 == 0 else (page - 9) * 1000
            csv_filename = f"Datos/{start_record}-{page*1000}_registros.csv"
            df = pd.DataFrame(all_data, columns=columns)
            df.to_csv(csv_filename, index=False)
            print(f"Datos guardados en {csv_filename}.")
            all_data = []  # Limpia los datos acumulados

        try:
            # Haz clic en el botón de "Siguiente página"
            next_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.mat-paginator-navigation-next"))
            )
            next_button.click()
            time.sleep(20)  # Aumenta el tiempo de espera para asegurar que la página cargue completamente
        except Exception as e:
            print(f"Error al navegar a la siguiente página en la página {page}: {str(e)}")
            break

# Encabezados de la tabla
columns = [
    "Convocante", "Convocatoria", "Código BDNS", "Reglamento", "Objetivos de la ayuda", 
    "Instrumentos de ayuda", "Tipo de Empresa", "Fecha de concesión", "Código de concesión", 
    "Fecha de registro", "Beneficiario", "Importe nominal", "Ayuda equivalente", 
    "Región concesión", "Sector de actividad NACE", "Referencia de la medida", 
    "Entidad art. 16 GBER", "Intermediario art. 21 GBER"
]

# Llamada a la función de extracción con un rango de páginas a extraer
start_page = 1  # Valor página inicial de la extracción
end_page = 410  # Valor página final de la extracción
go_to_page(start_page)  # Navegar a la página de inicio
extract_range(start_page, end_page)

# Cierra el navegador
driver.quit()

In [ ]:
# Especifica el directorio donde están los archivos CSV
directory = "Datos"

# Lista para almacenar los DataFrames de cada archivo CSV
dataframes = []

# Recorre todos los archivos en el directorio
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Une todos los DataFrames en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

# Elimina los registros duplicados (si es necesario)
unique_combined_df = combined_df.drop_duplicates()

# Muestra el número total de registros y los primeros registros
print(f"Número de registros únicos: {len(unique_combined_df)}")
print(unique_combined_df.head())

# Opcional: Guarda el DataFrame combinado en un archivo CSV
unique_combined_df.to_csv("Datos_extraidos_combinados.csv", index=False)